## Loading the test images

In [ ]:
import cv2
import matplotlib.pyplot as plt

# List with 5 pink roses and 5 non-pink roses
flowers_list = [
    "../raw_data/flowers/rose/9185768268_1e48d4d119_c.jpg",
    "../raw_data/flowers/rose/9230421241_665f294ca3_c.jpg",
    "../raw_data/flowers/rose/9692922135_ca989f7f65_c.jpg",
    "../raw_data/flowers/rose/8198408482_c7510b0992_c.jpg",
    "../raw_data/flowers/rose/8702677054_b916a65bf4_c.jpg",

    "../raw_data/flowers/rose/8739679581_9479ac63e4_c.jpg",
    "../raw_data/flowers/rose/813873375_32f738bf5a_c.jpg",
    "../raw_data/flowers/rose/7826516734_9e794be6af_c.jpg",
    "../raw_data/flowers/rose/6836630450_fb4f6eafc8_c.jpg",
    "../raw_data/flowers/rose/3622857482_8bb39d0a7f_c.jpg"
]



small_flowers_list = [
    "../raw_data/flowers/rose/3622856054_4d50ddd306_c.jpg",
    "../raw_data/flowers/rose/3662530669_a2bd246d28_c.jpg",
    "../raw_data/flowers/rose/3747927271_6955aa40b8_c.jpg",
    "../raw_data/flowers/rose/34796172892_d3c8ea4078_c.jpg",
    "../raw_data/flowers/rose/29133810912_5fbe8d34b3_c.jpg",

    "../raw_data/flowers/rose/185166267_cc52dccf4c_c.jpg",
    "../raw_data/flowers/rose/20018037665_61c39ffe84_c.jpg",
    "../raw_data/flowers/rose/15802683562_31d59c49c6_c.jpg",
    "../raw_data/flowers/rose/1086116212_bd2adc7d81_c.jpg",
    "../raw_data/flowers/rose/17891819940_853b7d932e_c.jpg"
]

#Define which test set to use:
# test_set = flowers_list
test_set = small_flowers_list


# Set up a subplot for 2 rows and 5 columns
fig, axs = plt.subplots(2, 5, figsize=(20, 8))

# Display all roses in a 2x5 grid
for i, flower in enumerate(test_set):
    # Load the image
    img = cv2.imread(flower)

    # Convert from BGR to RGB (OpenCV uses BGR by default)
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    # Calculate the row and column indices for subplot
    row = i // 5
    col = i % 5

    # Display the image in the appropriate subplot
    axs[row, col].imshow(img_rgb)
    axs[row, col].axis('off')  # Turn off axis numbers and ticks

# Adjust layout and display the plot
plt.tight_layout()
plt.show()


## Define function that returns upper and lower hue limits


In [ ]:
import numpy as np

def get_limits(color):
    """
    get_limits takes a color in BGR values and return upper and lower hue values.
    example: lowerHue, upperHue = get_limits([220, 140, 255])
    """

    c = np.uint8([[color]])  # BGR values
    hsvC = cv2.cvtColor(c, cv2.COLOR_BGR2HSV)

    #Get the hue value
    hue = hsvC[0][0][0]

    # Handle red hue wrap-around
    if hue >= 165:  # Upper limit for divided red hue
        lowerLimit = np.array([hue - 10, 100, 100], dtype=np.uint8)
        upperLimit = np.array([180, 255, 255], dtype=np.uint8)
    elif hue <= 15:  # Lower limit for divided red hue
        lowerLimit = np.array([0, 100, 100], dtype=np.uint8)
        upperLimit = np.array([hue + 10, 255, 255], dtype=np.uint8)
    else:
        lowerLimit = np.array([hue - 10, 100, 100], dtype=np.uint8)
        upperLimit = np.array([hue + 10, 255, 255], dtype=np.uint8)

    return lowerLimit, upperLimit


## Loading one Rose and printing the mask

In [ ]:
import numpy as np

#load pink rose image
img = cv2.imread(test_set[0])

#define pink in BGR color space
pink = [220, 140, 255]

#convert image from BGR into HSV color space
hsvImage = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)

#get limits of pink
lowerHue, upperHue = get_limits(pink)

#define mask
mask = cv2.inRange(hsvImage, lowerHue, upperHue)

plt.imshow(mask)
np.unique(mask, return_counts=True)


## Running color detection on all 10 test samples


In [ ]:
from IPython.display import display, Image
import os

# Set up a subplot for 2 rows and 5 columns
fig, axs = plt.subplots(2, 5, figsize=(20, 8))

#define pink in BGR color space
pink = [220, 140, 255]

# Get limits of pink
lowerHue, upperHue = get_limits(pink)

# Display all roses in a 2x5 grid
for i, flower in enumerate(test_set):
    # Load the image
    img = cv2.imread(flower)

    # Convert image from BGR into HSV color space
    hsvImage = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)

    # Define mask
    mask = cv2.inRange(hsvImage, lowerHue, upperHue)

    #Find unique values (black and white --> white = pink detected)
    unique, counts = np.unique(mask, return_counts=True)

    #threshold for amount of pink needed in the image
    pink_threshold = 0.005

    #check if pink was detected
    if counts.shape[0] == 2:
        if counts[1] / counts[0] > pink_threshold:
            print(f"{i} pic {flower}: {unique}, {counts} --> {counts[1] / counts[0]}")
        else:
            print(f"{i} pic {flower}: {unique}, {counts} --> not enough pink detected")
    else:
        print(f"{i} pic {flower}: {unique}, {counts} --> no pink detected")


    # Save the mask to a temporary file
    cv2.imwrite(f"mask_{i}.png", mask)

    # Calculate the row and column indices for subplot
    row = i // 5
    col = i % 5

    # Display the image using IPython's display function in the subplot
    axs[row, col].imshow(mask, cmap='gray')  # Display the mask using grayscale colormap
    axs[row, col].set_title(f'Flower {i+1}')  # Set a title for each subplot
    axs[row, col].axis('off')  # Turn off axis numbers and ticks

# Adjust layout
plt.tight_layout()
plt.show()

# Delete temporary images after the loop
for i in range(len(flowers_list)):
    file_path = f"mask_{i}.png"
    if os.path.exists(file_path):
        os.remove(file_path)


## Defining final function


In [ ]:
from IPython.display import display, Image

def find_pink(image_path):
    """
    find_pink takes the path to image as input and return True if pink was detected
    example: find_pink("raw_data/flowers/rose/9185768268_1e48d4d119_c.jpg")
    """
    #set default return value
    found_pink = False

    #Define pink in BGR color space
    pink = [220, 140, 255]

    #Get limits of pink
    lowerHue, upperHue = get_limits(pink)

    #Load the image
    img = cv2.imread(image_path)

    #Convert image from BGR into HSV color space
    hsvImage = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)

    #Define mask
    mask = cv2.inRange(hsvImage, lowerHue, upperHue)

    #Identify how much pink is in the image
    unique, counts = np.unique(mask, return_counts=True)

    #threshold for amount of pink needed in the image
    pink_threshold = 0.005

    #check if pink was detected
    if counts.shape[0] == 2:
        if counts[1] / counts[0] > pink_threshold:
            found_pink = True

    return found_pink


## Test function with flowers list


In [ ]:
for flower in test_set:
    print(find_pink(flower))
